In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing import image 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from tensorflow.image import rgb_to_grayscale

In [ ]:
def normal_nonnormal(x): 
    if x == 'Normal': 
        return x 
    else: 
        return 'Non-Normal'

df = pd.read_csv('dataset/Chest_xray_Corona_Metadata.csv')
na_fill = {'Label_1_Virus_category': 'Normal'}
df = df.fillna(value = na_fill)

df['BinaryCategory'] = df['Label_1_Virus_category'].map(normal_nonnormal)
df = df.join(pd.get_dummies(df.BinaryCategory.values, prefix = 'type')) # one hot
df = df[['ImagePath', 'BinaryCategory', 'type_Non-Normal', 'Dataset_type']] # Chỉ giữ các cột cần thiết

X = df[['ImagePath', 'BinaryCategory', 'Dataset_type']]
y = df[['type_Non-Normal']]

# --- KHÔNG SỬ DỤNG train_test_split NỮA ---
# Vì tập dữ liệu đã có cột 'Dataset_type' (train/test), ta sẽ sử dụng nó để chia.

x_train = X[X.Dataset_type == 'TRAIN'].drop('Dataset_type', axis=1)
y_train = y[X.Dataset_type == 'TRAIN']
x_test = X[X.Dataset_type == 'TEST'].drop('Dataset_type', axis=1)
y_test = y[X.Dataset_type == 'TEST']

# Loại bỏ cột nhị phân cho việc stratification/kiểm tra
x_train = x_train.drop('BinaryCategory', axis = 1)
x_test = x_test.drop('BinaryCategory', axis = 1)


def get_image_value(path):
    '''This function will retrive the RGB array for an image given its path'''
    img = image.load_img(path, target_size = (96,96,3))
    img = image.img_to_array(img)

    return img/255


def get_data(df, dataset_type):
    '''This function will retrive the paths for each item within a sample, and call get_image_value to retrieve
    the RGB array for each image. It uses the Dataset_type column for path construction.'''
    from tqdm import tqdm
    img_list = []
    # Cập nhật đường dẫn hình ảnh để trỏ đến thư mục 'train' hoặc 'test'
    # Dựa trên cột 'Dataset_type' ('TRAIN' -> 'train', 'TEST' -> 'test')
    for path in tqdm(df.ImagePath.values, desc = f'Gathering {dataset_type} Image Arrays'):
        # Đường dẫn mới: 'dataset/Corona-Chest-XRay-Dataset/{train|test}/{ImagePath}'
        # Giả định ImagePath chỉ là tên file.
        folder = 'train' if dataset_type == 'TRAIN' else 'test'
        full_path = f'dataset/Corona-Chest-XRay-Dataset/{folder}/{path}'
        img_list.append(get_image_value(full_path))
    return np.array(img_list).squeeze()

# Lấy dữ liệu
x_train_data = get_data(X[X.Dataset_type == 'TRAIN'], 'TRAIN')
x_test_data = get_data(X[X.Dataset_type == 'TEST'], 'TEST')

# Cập nhật x_train/x_test
x_train = x_train_data
x_test = x_test_data

Gathering TRAIN Image Arrays:   0%|          | 0/5286 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../dataset/Corona-Chest-XRay-Dataset/train/IM-0128-0001.jpeg'

In [ ]:
import pickle 
from imblearn.over_sampling import SMOTE


smote = SMOTE() 
print('Old Shape', x_train.shape)
x_train_resampled, y_train_resampled = smote.fit_sample(x_train.reshape(len(x_train), 28*28*3), y_train)
print('New Shape', x_train_resampled.shape)

smote
pickle.dump(x_train_resampled, open('pickles/SMOTE_x_train.p', 'wb'))
pickle.dump(y_train_resampled, open('pickles/SMOTE_y_train.p', 'wb'))

#normal 
pickle.dump(y_test, open('pickles/Normal_y_test.p', 'wb'))
pickle.dump(x_test, open('pickles/Normal_x_test.p', 'wb'))
pickle.dump(x_train, open('pickles/Normal_x_train.p', 'wb'))
pickle.dump(y_train, open('pickles/Normal_y_train.p', 'wb'))
